<a href="https://colab.research.google.com/github/Padinant/CodeFest-2025-Garbage-Classifier-Project/blob/main/ProjectExecutable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Team Info
Team # 10: Padina Nasiri Toussi, Timothy Winans

This colab notebook is the executable trains the Computer Vision model used for our team's Spring 2025 CodeFest project based on challenge #1

It aims to correctly classify garbage images into 10 categories for the purpose of recycling

For our final model, we have used the pretrained MobileNet model -known for its efficiency and low-cost performance- as base model, which we finetuned on our data.

The model has been trained on the dataset found here:
https://www.kaggle.com/datasets/sumn2u/garbage-classification-v2?resource=download

We have named our project GAIA, after the ancient Greek goddess of nature and all life - which we believe is an apt name as our project aims to give recyclable waste a chance of a new life.

Our model is called Oscar, The G.A.I.A (Green Automated Intelligent Assistant) Garbage Detector - Which we think is a whimsical name, while still being on theme and carrying our project name as an acronym.


# Why is this Eco-Friendly?
- Efficiently trained model, trained on a few epochs
- An already energy efficient and low cost base model (MobileNet)
- Could be deployed on low-power devices like Raspberry Pi or mobile phones
- The dataset is processed efficiently, using image compression and resizing techniques to limit storage and computational overhead.
- Batch processing and on-the-fly augmentation reduce the need for high memory and storage requirements.
- We aim to minimize contaminated and missed recyclings by focusing on our model's percision and recall rates, alongside its accuracy

# Info
The Executable Google Colab Notebook for Team 10 - Spring 2025 CodeFest

Members:
- Padina Nasiri Toussi
- Timothy Winans

# Instructions for Execution
This section covers instructions on how to use this Google Colab file.

In this notebook, you can import our fine-tuned computer vision CNN model, and utilize it in producing predictions for the recycling category of input images.

You can import the model in the 'Importing the Model' section.

2 main functions have been provided in 'Main Functions' which allow you to use the model to predict the recycling category of 1 or multiple images.

There are also some helper function in Useful Helper Functions which you could also use to help preprocess your input data, or even download our original dataset from kaggle

# How to Run our Garbage Detector Model on this Google Colab

This guide provides a step-by-step approach to running our fine-tuned MobileNet Garbage Detector notebook on Google Colab.

## Step 0: Have the project's shared Google Drive folder called Final_Submission handy
This is where our final trained models can be found

## Step 1: Run the cell under 'Importing Some Useful Libraries'
This imports libraries used throughout the notebook

## Step 2: Run the cells under 'Useful Constants and Variables'
A few variables used throught the notebook

## Step 3: Import model from the Final_Submission folder to Google Colab

Option 1: Open from Local Machine

Option 2: Open Model from Google Drive (Recommended)

for more instructions on these options see 'Importing the Model'

## Step 4: Run the cells under 'Useful Helper Functions'
These are some helper functions, but they may be also of use to you in preparing input data for the model

## Step 5: Run the cells under 'Main Functions'
You can call these functions directly to get our model's predictions on input images, in the next step.

## Step 6: Testing
In 'Test Section', feel free to write any code to evaluate and play with our model

# Importing Some Useful Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
from PIL import Image
import os
import cv2
import imghdr
import glob
from itertools import islice

# ensure any matplotlib graphs will be displayed inline
%matplotlib inline

import keras
import tensorflow as tf
from tensorflow.keras import datasets
from tensorflow.keras.models import load_model

from sklearn.model_selection import train_test_split

<ipython-input-7-3ecb90bb14c8>:9: DeprecationWarning: 'imghdr' is deprecated and slated for removal in Python 3.13
  import imghdr


# Useful Constants and Variables
These variables will later help in building our main functions

In [ ]:
# define global variable - CATEGORY_NAMES (array of len 10)
CATEGORY_NAMES = ['battery', 'biological', 'cardboard', 'clothes', 'glass', 'metal', 'paper', 'plastic', 'shoes', 'trash']

In [ ]:
# resizing variables - these will help in converting images into model input
img_width = 250
img_height = 250
img_size = (img_width, img_height)

# Importing the Model
There are 2 options to import the model: offline through upload to the colabatory, and online through mounting Google Drive

The online option requires you to link your google Drive account to this colabatory. If you prefer not to do that, please follow the instructions for the offline option.



## Option 1: Offline Model Import
Instructions:

Navigate to the shared folder submitted by the team (named: Final_Submission). From there, download the keras format of the model (titled: MobileNetGarbageDetector.keras) into your local computer.

Then, navigate to the files section of this colabatory (on the left side of the screen), and 'press the upload to session storage button'. From there, you can upload our keras model into the session storage files.

After that, run the code below.

In [ ]:
# importing the model from the notebook session storage
base_path = '/content/'
model_name = 'MobileNetGarbageDetector1_9.keras'
retrieval_path = base_path + model_name

model = load_model(retrieval_path)

## Option 2: Online Model Import

In [ ]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True) # this might prompt a pop-up from Google!

# add your retrieval path here
base_path = '/content/drive/'
model_name = 'MobileNetGarbageDetector1_9.keras'
retrieval_path = base_path + model_name

model = load_model(retrieval_path)

# Useful Helper Functions
These functions will later help in building our main functions, or can be useful to the user when preparing data to evaluate our model with

In [ ]:
# Some useful kaggle-related functions

def install_kaggle():
  # installs the kaggle API client in the notebook
  !pip install kaggle


def download_kaggle_dataset(kaggle_username: str, kaggle_userkey: str):
  """ Takes in the user's kaggle information as input, and returns the downloaded
  garbage dataset which was used for this challenge.
  The dataset will be downloaded from:
    https://www.kaggle.com/datasets/sumn2u/garbage-classification-v2?resource=download

  The function unzips the dataset and loads it into the notebook's session storage
  """

  os.environ['KAGGLE_USERNAME'] = kaggle_username  # kaggle username
  os.environ['KAGGLE_KEY'] = kaggle_userkey # kaggle key

  # Download the garbage dataset
  !kaggle datasets download -d sumn2u/garbage-classification-v2

  # Unzip the downloaded dataset
  !unzip garbage-classification-v2.zip

In [ ]:
# Functions relating to image quirks

def drop_unexpected_images(base_path = '/content/garbage-dataset/', folder_names = CATEGORY_NAMES):
  # Removing any unexpected files (anything not JPEG, PNG, JPG)
  # Prerequisite: The test images have been stored in inner folders, whose names match folder_names
  # Prerequisite: Base path is the path to an outer folder, storing 1 or more inner category folders containing images
  # this function can be useful as there are a few images in the original dataset which might need to be cleaned

  count = 0
  for folder_name in folder_names:
    folder_path = base_path + folder_name

    all_files = glob.glob(os.path.join(folder_path, "*"))

    for file in all_files:
      file_type = imghdr.what(file)  # Detect file format
      if file_type not in ["jpeg", "png", "jpg"]:
          print(f"Unknown image type: {file} (Detected as {file_type})")

          # Delete file from the dataset
          os.remove(file)
          print(f"{file} was successfully removed from dataset")
          count += 1

  print(f"{count} images have been removed from dataset.")


# Several useful functions to remvoe transparency from an image, or image path(s)
def remove_transparency_from_image(img, bg_color=(255, 255, 255)):
  """
  This function removes the alpha channel (quantifying opaqueness/transparency) of an image, if the channel exists.
  It is given the actual image, and the background color as input, and returns the updated image
  bg_color is the RGB background color the value of the alpha channel will be replaced and blended with
  By default, bg_color has been set to white, but you can also set it to black, or any other color
  Rewrites the image to only have the 3 RGB channels
  """
  if img.shape[-1] == 4:  # Has an alpha channel
    alpha = img[:, :, 3] / 255.0  # Normalize alpha to 0-1
    img_rgb = img[:, :, :3]  # Get RGB channels

    # Blend with white background --> ie: opaque pixels are unaffected, transparent pixels become white
    img_new = (1. - alpha[:, :, None]) * np.array(bg_color, dtype=np.float32) + alpha[:, :, None] * img_rgb
    img_new = img_new.astype(np.uint8)

    return img_new  # return updated image

  return img

def remove_transparency(image_path, bg_color=(255, 255, 255)):
  """
  This function removes the alpha channel (quantifying opaqueness/transparency) of an image if the channel exists.
  It is given the image path and background color as input, and doesn't return anything
  bg_color is the RGB background color the value of the alpha channel will be replaced and blended with
  By default, bg_color has been set to white, but you can also set it to black, or any other color
  Rewrites the image to only have the 3 RGB channels
  """
  img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)  # Load image with all channels (including alpha)
  if img.shape[-1] == 4:  # Has an alpha channel
    alpha = img[:, :, 3] / 255.0  # Normalize alpha to 0-1
    img_rgb = img[:, :, :3]  # Get RGB channels

    # Blend with white background --> ie: opaque pixels are unaffected, transparent pixels become white
    img_new = (1. - alpha[:, :, None]) * np.array(bg_color, dtype=np.float32) + alpha[:, :, None] * img_rgb
    img_new = img_new.astype(np.uint8)

    cv2.imwrite(image_path, img_new)  # Overwrite the image

def remove_transparency_from_folders(base_path = '/content/garbage-dataset/', folder_names = CATEGORY_NAMES, target_image_type = "jpg"):
  """
  This function applies remove_transparency to all images of a given type inside the session storage

  Prerequisites:
  # Base path is the path to an outer folder, storing 1 or more inner category folders containing images
  # The test images have been stored in inner folders, whose names match folder_names
  # target_image_type has to be in ["jpeg", "png", "jpg"]
  """
  for folder_name in folder_names:
    folder_path = base_path + folder_name

    image_paths = glob.glob(os.path.join(folder_path, "*." + target_image_type))  # Find all instances of the target type
    print(f"Processing {len(image_paths)} images in {folder_path}...")

    # applying remove_transparency to each file
    for file in image_paths:
      remove_transparency(file)

  print(f"All transparent images of type {target_image_type} have been processed.")

# Main Functions
These functions can be called by the user to provide functionality, and test our model

In [ ]:
def image_to_category(img, output_type = 'single_category'):
  """
  # Takes in an image as input, and returns the predicted category
  # Precondition: the image type (as determined by imghdr.what()) must be one of JPG, JPEG, PNG
  # Precondition: the variable output_type may be one of the following options :
    # 'single_category': returns the top predicted category, and the corresponding predicted percentage
    # 'top_three': returns the top three predicted categories in order of likelihood, and the corresponding percentage
    # 'all_categories': returns the predicted percentage for all 10 possible categories
  # Postcondition: the output is a dictionary of type {string: float}.
  # The dictionary will store {category_name: prediction_value} and be ordered in order of how high the predicted value was

  # Function Explanation
  # step 0: A precondition is that the image has to be of an acceptable file format (PNG, JPG, JPEG)
  # step 1: remove transparency (alpha channel) from image
  # step 2: convert to correct input format
  # step 3: get model output
  # step 4: use the results to create a dictionary
  # step 5: return dictionary
  """

  # A bit of setup
  if output_type not in ('single_category', 'top_three', 'all_categories'):
    output_type = 'single_category'
  image = img.copy  # to avoid modifying the original image

  # step 1: remove transparency (alpha channel) from image
  remove_transparency_from_image(image)

  # step 2: convert to correct input format (tf.data.Dataset)
  image = tf.image.resize(image, img_size)

  image = tf.expand_dims(image, axis=0) # Add a new batch dimension to the front of the image
                                        # image is now of shape: [1, img_width, img_height, 3]
  input_dataset = tf.data.Dataset.from_tensor_slices(image)

  # step 3: get model output
  predictions = model.predict(input_dataset) # prediction is an array of len 10 corresponding to the different categories

  # step 4a: use the results to create a dictionary corresponding to categories
  classifications = {}
  for i in range(0, 10):
    classifications[CATEGORY_NAMES[i]] = predictions[i] # {category_name: predicted_value}

  # step 4b: reorder the dictionary based on how high the prediction for each category was
  sorted_classifications = dict(sorted(classifications.items(), key=lambda item: item[1], reverse=True))

  # step 5: return (the appropriate version) of the dictionary
  if output_type == 'single_category':
    return dict(islice(sorted_classifications.items(), 1))  # Using islice to get the first key value pair of the dict
  elif output_type == 'top_three':
    return dict(islice(sorted_classifications.items(), 3))  # Using islice to get the first 3 key value pairs of the dict
  else:
    return sorted_classifications # no need to change anything


In [ ]:
def image_folder_to_category(folder_path, output_type = 'single_category'):
  """
  # Takes in a folder path as input and returns a list of predicted categories
  # Precondition: folder_path has to be the path to a folder in session storage, containing image files
  # Precondition: The type of those images (as determined by imghdr.what()) must be in (JPG, JPEG, PNG)
  # Precondition: the variable output_type may be one of the following options:
    # 'single_category': Each dictionary includes the top predicted category, and the corresponding predicted percentage, for corresponding image
    # 'top_three': Each dictionary includes the top three predicted categories in order of likelihood, and the corresponding percentage, for corresponding image
    # 'all_categories': Each dictionary includes the predicted percentage for all 10 possible categories, for corresponding image
  # Postcondition: the output is a list of dictionaries of type {string: float}.
  # The order of the elements in the list, will correspond to that of the images in the folder
  # Each inner dictionary will store {category_name: prediction_value}
  # Each inner dictionary will be internally ordered in in terms of size of prediction value for each category

  # Function Explanation
  # step 0: drop unexpected files
  # step 1: remove transparency (alpha channel) from images
  # step 2: convert to correct input format
  # step 3: get model output
  # step 4: use the results to create the final list
  # step 5: return list
  """
  # A bit of setup
  if output_type not in ('single_category', 'top_three', 'all_categories'):
    output_type = 'single_category'

  # step 0: drop unexpected file formats
  drop_count = 0
  all_files = glob.glob(os.path.join(folder_path, "*"))

  for file in all_files:
    file_type = imghdr.what(file)  # Detect file format
    if file_type not in ["jpeg", "png", "jpg"]:
        print(f"Unknown image type: {file} (Detected as {file_type})")

        # Delete file from the dataset
        os.remove(file)
        print(f"{file} was successfully removed from dataset")
        drop_count += 1

  print(f"{drop_count} images have been removed from dataset.")

  # step 1: Remove transparency
  for target_image_type in ("jpeg", "png", "jpg"):
    image_paths = glob.glob(os.path.join(folder_path, "*." + target_image_type))  # Find all instances of the target type

    print(f"Processing {len(image_paths)} images of type {target_image_type} in {folder_path}...")

    # applying remove_transparency to each file
    for file in image_paths:
      remove_transparency(file)

  print(f"All transparent images of type {target_image_type} have been processed.")


  # step 2: convert to correct input format (tf.data.Dataset)
  input_dataset = tf.keras.utils.image_dataset_from_directory(
    directory=folder_path,
    image_size=img_size,  # Resize images
    batch_size=32,
    shuffle=False,
    seed=25,  # Seed is not needed, since we're not shuffling
    )

  # step 3: get model output
  folder_predictions = model.predict(input_dataset) # prediction is an array of len 10 corresponding to the different categories

  # step 4: create the return list
  result = []
  for predictions in folder_predictions:
    # step 4a: use the results to create a dictionary corresponding to categories
    classifications = {}
    for i in range(0, 10):
      classifications[CATEGORY_NAMES[i]] = predictions[i] # {category_name: predicted_value}

    # step 4b: reorder the dictionary based on how high the prediction for each category was
    sorted_classifications = dict(sorted(classifications.items(), key=lambda item: item[1], reverse=True))

    # step 5: return (the appropriate version) of the list
    if output_type == 'single_category':
      result.append(dict(islice(sorted_classifications.items(), 1)))  # Using islice to get the first key value pair of the dict
    elif output_type == 'top_three':
      result.append(dict(islice(sorted_classifications.items(), 3)))  # Using islice to get the first 3 key value pairs of the dict
    else:
      result.append(sorted_classifications) # no need to change anything

  return result

# Test Section

Here, you can experiment with the provided functions (and even the dataset), to evaluate our model


In [ ]:
# Feel free to add more code!

# Thank You

Thank you for viewing our code.

Have a nice day!

This project is open-source under the MIT License.